<a href="https://colab.research.google.com/github/totminaekaterina/RUSSE-2022-Detoxification/blob/main/finetune_rugpt3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import argparse
import json
import random
from pprint import pprint
from pathlib import Path
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          Trainer,
                          TrainingArguments,
                          TrainerCallback,
                          AdamW,
                          get_linear_schedule_with_warmup)

TypeError: expected string or bytes-like object

In [3]:
!pip install transformers

In [2]:
def set_random_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

In [3]:
def get_random_example(dataset):
    sample = dataset.sample()
    prompt = f'<|startoftext|>{sample["sentence1"].item()}<|sep|>'
    true_output = sample["sentence2"].item()
    return prompt, true_output

In [4]:
class SimplificationDataset(torch.utils.data.Dataset):
    def __init__(self, texts_list, tokenizer, gpt2_type="gpt2", max_length=1024):
        self.tokenizer = tokenizer

        texts_combined = []
        for input_text, out_text in texts_list:
            text_combined = f"<|startoftext|>{input_text}<|sep|>{out_text}<|endoftext|>"
            texts_combined.append(text_combined)
        self.encodings = tokenizer(texts_combined,
                              truncation=True,
                              max_length=max_length,
                              padding="max_length",
                              return_tensors="pt")

    def __len__(self):
        return len(self.encodings.input_ids)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["label"] = item["input_ids"]
        return item

In [5]:
class PrintExampleCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, logs=None, **kwargs):
        prompt, true_output = get_random_example(valid)
        print(prompt.strip("<|startoftext|>").strip("<|sep|>"), true_output, sep="\n")
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
        model.eval()
        with torch.no_grad():
            sample_outputs = model.generate(
                input_ids,
                do_sample=True,   
                top_k=50,
                max_length=MAX_LENGTH,
                top_p=0.95,
                temperature=0.9,
                num_return_sequences=1
            ).detach().cuda()
        model.train()

        for sample in sample_outputs:
            res = (tokenizer.decode(sample, skip_special_tokens=False)
                            .split("<|sep|>")[1]
                            .replace("<|pad|>", "")
                            .replace("<|endoftext|>", ""))
            print(res, "-" * 80, sep="\n")

In [6]:
TRAIN_PATH = "NLI_train.csv"
VALID_PATH = "NLI_val.csv"
TEST_PATH = "NLI_test.csv"

In [7]:
train = pd.read_csv(TRAIN_PATH, encoding='utf-8-sig', sep=';')
valid = pd.read_csv(VALID_PATH, encoding='utf-8-sig', sep=';')
valid.drop(["label"], axis=1, inplace=True)
train.drop(["label"], axis=1, inplace=True)
valid.columns = ["sentence1", "sentence2"]

In [8]:
train.head()

,sentence1,sentence2
0,В реанимации: измеренные жиз. показатели Т 37....,Пациент гемодинамически нестабилен.
1,63-летняя женщина с гиперхолестеринемией и язв...,у пациента отстутствует история болезни
2,ИСТОРИЯ ТЕКУЩЕГО ЗАБОЛЕВАНИЯ: Пациент - мужчин...,у пациента нормальная функция почек
3,ИСТОРИЯ: [**Фамилия пациента**] - мальчик 33-6...,пациент доношен
4,"Отрицал наличие головной боли, тошноты или рвоты.",У него гематемезис


# load model

In [9]:
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# add special tokens

In [14]:
special_tokens = {
    "bos_token": "<|startoftext|>",
    "pad_token": "<|pad|>",
    "sep_token": "<|sep|>",
}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

Embedding(50261, 768)

In [15]:
MAX_LENGTH = 200
DATA_COLS = ["sentence1", "sentence2"]
train_dataset = SimplificationDataset(train[DATA_COLS].values.tolist(), tokenizer, max_length=MAX_LENGTH)
valid_dataset = SimplificationDataset(valid[DATA_COLS].values.tolist(), tokenizer, max_length=MAX_LENGTH)

EPOCH_STEPS = len(train_dataset) // 8 // 1
EVAL_STEPS = EPOCH_STEPS
print(f"Total steps: {EPOCH_STEPS * 5}\nEvaluate and save every {EVAL_STEPS} steps.")

Total steps: 3120
Evaluate and save every 624 steps.


In [16]:
training_args = TrainingArguments(
    output_dir="result_rugpt3small_NLI",
    logging_dir="logs_rugpt3small_NLI",
    logging_first_step=True,
    num_train_epochs=50,
    evaluation_strategy="steps",
    eval_steps=EVAL_STEPS,
    save_steps=EVAL_STEPS,
    logging_steps=100,
    lr_scheduler_type="linear",
    warmup_steps=500,
    learning_rate=0.00005,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    weight_decay=0,
    fp16=False,
    seed=19,
)

In [17]:
Path(training_args.output_dir).mkdir(exist_ok=True)
Path(training_args.logging_dir).mkdir(exist_ok=True)

In [18]:
pprint(training_args.to_dict())

{'adafactor': False,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'auto_find_batch_size': False,
 'bf16': False,
 'bf16_full_eval': False,
 'data_seed': None,
 'dataloader_drop_last': False,
 'dataloader_num_workers': 0,
 'dataloader_pin_memory': True,
 'ddp_bucket_cap_mb': None,
 'ddp_find_unused_parameters': None,
 'ddp_timeout': 1800,
 'debug': [],
 'deepspeed': None,
 'disable_tqdm': False,
 'do_eval': True,
 'do_predict': False,
 'do_train': False,
 'eval_accumulation_steps': None,
 'eval_delay': 0,
 'eval_steps': 624,
 'evaluation_strategy': 'steps',
 'fp16': False,
 'fp16_backend': 'auto',
 'fp16_full_eval': False,
 'fp16_opt_level': 'O1',
 'fsdp': [],
 'fsdp_config': {'fsdp_min_num_params': 0,
                 'xla': False,
                 'xla_fsdp_grad_ckpt': False},
 'fsdp_min_num_params': 0,
 'fsdp_transformer_layer_cls_to_wrap': None,
 'full_determinism': False,
 'gradient_accumulation_steps': 1,
 'gradient_checkpointing': False,
 'greater_is_better'

In [19]:
with open("run_parameters.txt", "w") as f:
  pprint(training_args.to_dict(), f)

trainer = Trainer(
    model=model.to(device),
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    callbacks=[PrintExampleCallback],
)

trainer.train()

C:\Users\totmi\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\totmi\AppData\Local\Temp\ipykernel_11336\2097516035.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
624,0.491100,0.924235
1248,0.390000,0.994994
1872,0.305100,1.089051
2496,0.244400,1.141434
3120,0.198000,1.186871
3744,0.159200,1.222141
4368,0.134100,1.250471
4992,0.111800,1.280737
5616,0.099200,1.305379
6240,0.090200,1.319237


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Боксер Денис Лебедев не исключил, что в команде Александра Поветкина есть предатель, который мог «подсыпать» ему допинг. Напомним, ранее стало известно, что перед боем за титул временного чемпиона мира по версии Всемирного боксерского совета (WBC) с канадцем Бермейном Стиверном в допинг-пробе россиянина был обнаружен запрещенный препарат остарин. В результате бой был отменен, так как соперник отказался его проводить.
 Поветкина точно предали.
 Бывший боксер российского чемпиона по версии Всемирного боксерского совета (WBC) на профессиональном ринге стал новым чемпионом мира по версии Всемирного боксерского совета (WBC).
--------------------------------------------------------------------------------


C:\Users\totmi\AppData\Local\Temp\ipykernel_11336\2097516035.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Пассажир попросил убрать бутылку, но замечание разозлило выпивающих. Один из них стал вести себя очень агрессивно. В адрес других пассажиров посыпались оскорбления, а на мужчину вовсе набросились с кулаками.
 Пассажир просил выпивающих поделиться выпивкой.
 Пассажиры не замечали нападений.
--------------------------------------------------------------------------------


C:\Users\totmi\AppData\Local\Temp\ipykernel_11336\2097516035.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ситуация по заболеваемости гриппом и острыми респираторными вирусными инфекциями оценивается как напряженная, наблюдаются сезонный рост заболеваемости.
 Заболеваемость гриппом не растет в этом сезоне.
 Температура гриппа и острых респираторных вирусных инфекций в норме.
--------------------------------------------------------------------------------


C:\Users\totmi\AppData\Local\Temp\ipykernel_11336\2097516035.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=15650, training_loss=0.1859611633410469, metrics={'train_runtime': 8565.5525, 'train_samples_per_second': 29.181, 'train_steps_per_second': 1.827, 'total_flos': 2.551169664e+16, 'train_loss': 0.1859611633410469, 'epoch': 50.0})

In [20]:
trainer.save_model('NLI_for_neg_gen')